# xpander.ai single agent using Gemini (OpenAI sdk)

In [ ]:
from xpander_sdk import XpanderClient, LLMProvider
from openai import OpenAI
from dotenv import load_dotenv
from os import environ

load_dotenv()


In [2]:
GEMINI_API_KEY = environ["GEMINI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID_SINGLE"]

## Initialize xpander.ai Client

In [3]:
xpander_client = XpanderClient(api_key=XPANDER_API_KEY)

## Initialize OpenAI Client

In [4]:
gemini_openai_client = OpenAI(api_key=GEMINI_API_KEY,base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

## Load agent

In [ ]:
agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)

## Invoke the agent with query and create an execution task

In [ ]:
agent.add_task("""
Find employees of xpander.ai.
""")

## Init the agent's memory with input (query) and instructions

In [7]:
agent.memory.init_messages(input=agent.execution.input_message,instructions=agent.instructions)

## Run the agent and wait for result

In [ ]:
while not agent.is_finished():
    response = gemini_openai_client.chat.completions.create(
                model= "gemini-2.0-flash",
                messages=agent.messages,
                tools=agent.get_tools(llm_provider=LLMProvider.GEMINI_OPEN_AI),
                tool_choice=agent.tool_choice,
                temperature=0.0
        )
            
    # add messages directly from the LLM response
    agent.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump(),llm_provider=LLMProvider.GEMINI_OPEN_AI)
    # run tools
    agent.run_tools(tool_calls=tool_calls)


## Fetch & print the agent execution result

In [ ]:
execution_result = agent.retrieve_execution_result()

print("status", execution_result.status)
print("result", execution_result.result)